In [1]:
import matplotlib
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
from matplotlib.widgets import Button
import numpy as np
from skimage.io import imread, imsave
import json
import sys
sys.path.append(r'C:\Users\AhrensLab\Documents\GitHub\dmd')
from dmd.gui import DraggableRectangle, RoiDrawing
plt.ion()

In [3]:
calibration_data_fname = r'C:\Users\Ahrenslab\Documents\Github\dmd\dmd_calibration.json'
calibration_image_fname = r'Z:/davis/data/dmd_calibration/20170810/checkerboard_0.png'
calibration_image = imread(calibration_image_fname)

## Load calibration image from file and estimate position of DMD pattern 
# todo: automate this

In [6]:
from matplotlib.patches import Rectangle
fig, axs = plt.subplots(figsize=(8,8))
from skimage.exposure import adjust_gamma
axs.imshow(adjust_gamma(calibration_image,.1), origin='lower')

# Empirically, this is a pretty good guess, in case we can't load the calibration data file
# x_guess, y_guess, w_guess, h_guess = 230, 588, 1600, 874

with open(calibration_data_fname, 'r') as infile:
    calibration_data = json.load(infile)

x_guess, y_guess, w_guess, h_guess = calibration_data['dmd_rect']

# dmd_rect stores the coordinates of the corners of the rectangle instead of corner_x, corner_y, w, h
w_guess -= x_guess
h_guess -= y_guess

rect = Rectangle((x_guess,y_guess), w_guess, h_guess, fill=False)
axs.add_patch(rect)
dr = DraggableRectangle(rect)
dr.connect()
plt.show()

In [7]:
## TODO:Add a crosshair down the middle of the alignment rectangle

In [7]:
x, y, w, h = dr.rect.get_x(), dr.rect.get_y(), dr.rect.get_width(), dr.rect.get_height()
dmd_rect = [int(x), int(y), int(x + w), int(h + y)]
calibration_data = {'dmd_rect' : dmd_rect}
with open(calibration_data_fname, 'w') as outfile:
    json.dump(calibration_data, outfile)

print(x,y,w,h)

218.0 586.666015689 1631.0 891.0


## Load target image from file and draw ROI

In [9]:
sample_image_fname = r'Z:/davis/data/dmd_calibration/20170802/fish_1.png'
# todo: correctly infer the true position of the camera roi on the chip 
sample_image = imread(sample_image_fname)
full_cam = np.zeros((2048,2048))
corner = (1024 // 2, 0) 
#full_cam[corner[0]:corner[0] + sample_image.shape[0], corner[1] : corner[1] + sample_image.shape[1]] = sample_image
#sample_image = full_cam
data = sample_image[dmd_rect[1]:dmd_rect[3],dmd_rect[0]:dmd_rect[2]]

fig = plt.figure()
ax = plt.axes([.05, .05, .6, .6])
ax.set_title('Draw ROI')
ax.imshow(data, cmap='viridis')
#
roi_draw = RoiDrawing(ax, data)

ax_butt = plt.axes([0.7, 0.05, 0.1, 0.075])
wipe_butt = Button(ax_butt, 'Wipe')
wipe_butt.on_clicked(roi_draw.wipe)

ax_decr = plt.axes([.7, .15, .1, .075])
decr_button = Button(ax_decr, 'Prev ROI')
decr_button.on_clicked(roi_draw.focus_decr)

ax_incr = plt.axes([.7, .25, .1, .075])
incr_button = Button(ax_incr, 'Next ROI')
incr_button.on_clicked(roi_draw.focus_incr)
plt.show()

In [14]:
fig, axs = plt.subplots()
masks = np.array([r.get_mask() for r in roi_draw.rois if len(r.x) > 0])
plt.imshow(masks.sum(0), cmap='gray')
plt.show()

In [15]:
# Apply transformations to ROI images and save to disk
from skimage.transform import resize
dmd_dims = (684,608)
masks_resized = np.array([resize(m, output_shape = dmd_dims, mode='constant', cval=0) for m in masks])
# need to flip along x axis
masks_resized = masks_resized[:,:,::-1]
fig, axs = plt.subplots()
plt.imshow(masks_resized.sum(0),cmap='gray')
plt.show()

## Set output directory for ROI

In [16]:
out_path = r'C:/davis/20170802/20170802_7dpf_cy252xcy293_f1_opto_1/'

# Warning: this overwrites any existing ROI!!

In [17]:
for ind, m in enumerate(masks_resized):
    imsave(out_path + 'pattern_{:04d}.png'.format(ind), m)

C:\ProgramData\Anaconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint16
  .format(dtypeobj_in, dtypeobj_out))
